In [1]:
import sys
sys.path.append('../src')
from utils import EventStream
from cache_policy_buffers import CachePolicy
from model_buffers import OcdfgBuffer, OcpnBuffer
from model_builder_ocdfg import OcdfgModel, get_ocdfg_accuracy, visualize_ocdfg_overlap, get_ocdfg_avg_scores
from model_builder_ocpn import OcpnModel, get_ocpn_accuracy, get_ocpn_avg_scores

import matplotlib as mpl
import matplotlib.cm as cm
import numpy as np
from pathlib import Path

#### Loading `SmallExample.json` stream and preparing visualization

In [2]:
# Load event stream
event_stream = EventStream('../data/SmallExample.json', o2o_has_time=True)

# Define colors for object types
ots = sorted(event_stream.object_types)
ot_rgb_colors = cm.jet(np.linspace(0, 1, len(ots)))
ot_to_rgb_color = dict(zip(ots, ot_rgb_colors))
ot_to_hex_color = {ot: mpl.colors.rgb2hex(ot_rgb) for ot, ot_rgb in ot_to_rgb_color.items()}

Parsing ../data/SmallExample.json...
# of (removed) E2O target objects w/o type: 0
# of (removed) O2O target objects w/o type: 0
# events:			5
# object updates:		0
# O2O relations:		1
# E2O-derived O2O relations:	11
Enriching enabled: False
Finished parsing ../data/SmallExample.json in 0.00 min.


#### Discovery, visualization and evaluation of OC-DFG

In [3]:
# Discover and draw full OC-DFG
ocdfg_buf = OcdfgBuffer(
    50,
    50,
    CachePolicy.FIFO
)
ocdfg_buf.process_stream(event_stream.stream)
ocdfg_model = OcdfgModel(ocdfg_buf)
ocdfg_model_offl = OcdfgModel('../data/SmallExample.json')
ocdfg_model_offl.visualize(Path('../figures/SmallExample'), 'ocdfg_offl.pdf', ot_to_hex_color, visualize_dfgs=False)


In [4]:
# Print evaluation metrics for discovered OC-DFG
def print_ocdfg_scores(onl_model : OcdfgModel, offl_model : OcdfgModel) -> None:
    score_dict = get_ocdfg_accuracy(offl_model, onl_model)
    print('Total evaluation scores\n-----------------------')
    for key, val in score_dict.items():
        print(f'{key}:\t{val}')

    avg_score_dict = get_ocdfg_avg_scores(offl_model, onl_model)
    print(f'\nAverage evaluation scores\n-------------------------')
    for key, val in avg_score_dict.items():
        print(f'{key}:\t{val}')

In [5]:
# Evaluate online model with sufficiently large buffers to fit entire SmallExample log
print_ocdfg_scores(ocdfg_model, ocdfg_model_offl)

Total evaluation scores
-----------------------
node recall:	1.0
node accuracy:	1.0
node precision:	1.0
arc recall:	1.0
arc accuracy:	1.0
arc precision:	1.0
total node freq. MAE:	0.0
arc freq. MAE:	0.0
source/sink recall:	1.0
source/sink accuracy:	1.0
source/sink precision:	1.0
source/sink arc recall:	1.0
source/sink arc accuracy:	1.0
source/sink arc precision:	1.0

Average evaluation scores
-------------------------
recall:	1.0
precision:	1.0
accuracy:	1.0


In [6]:
# Visualize effect of using insufficiently small model buffers
ocdfg_buf_too_small = OcdfgBuffer(
    5,
    5,
    CachePolicy.FIFO
)
ocdfg_buf_too_small.process_stream(event_stream.stream)
ocdfg_model_too_small = OcdfgModel(ocdfg_buf_too_small)
visualize_ocdfg_overlap(ocdfg_model_offl, ocdfg_model_too_small, Path('../figures/SmallExample'), 'ocdfg_too_small_overlap.pdf', ot_to_hex_color)

# Evaluate online model with insufficiently small model buffers to fully capture SmallExample
print_ocdfg_scores(ocdfg_model_too_small, ocdfg_model_offl)


Total evaluation scores
-----------------------
node recall:	0.75
node accuracy:	0.75
node precision:	1.0
arc recall:	0.5
arc accuracy:	0.875
arc precision:	1.0
total node freq. MAE:	0.3333333333333333
arc freq. MAE:	0.0
source/sink recall:	0.8
source/sink accuracy:	0.8
source/sink precision:	1.0
source/sink arc recall:	0.6
source/sink arc accuracy:	0.625
source/sink arc precision:	0.75

Average evaluation scores
-------------------------
recall:	0.6625
precision:	0.9375
accuracy:	0.7625


#### Discovery, evaluation and visualization of OCPN

In [7]:
# Discover and draw full OCPN
ocpn_buf = OcpnBuffer(
    50,
    50,
    50,
    CachePolicy.FIFO,
    use_mixed_ocdfg_buf=True
)
ocpn_buf.process_stream(event_stream.stream)
ocpn_model = OcpnModel(ocpn_buf)
ocpn_model_offl = OcpnModel('../data/SmallExample.json')
ocpn_model_offl.visualize(Path('../figures/SmallExample'), 'ocpn_offl.pdf', ot_to_hex_color)

In [8]:
# Print evaluation metrics for discovered OCPN
def print_ocpn_scores(onl_model : OcpnModel, offl_model : OcpnModel) -> None:
    score_dict = get_ocpn_accuracy(offl_model, onl_model)
    print('Total evaluation scores\n-----------------------')
    for key, val in score_dict.items():
        print(f'{key}:\t{val}')

    avg_score_dict = get_ocpn_avg_scores(offl_model, onl_model)
    print(f'\nAverage evaluation scores\n-------------------------')
    for key, val in avg_score_dict.items():
        print(f'{key}:\t{val}')

In [9]:
# Evaluate online model with sufficiently large buffers to fit entire SmallExample log
print_ocpn_scores(ocpn_model, ocpn_model_offl)

Total evaluation scores
-----------------------
act-place-act recall:	1.0
act-place-act accuracy:	1.0
act-place-act precision:	1.0
double-arc recall:	1.0
double-arc accuracy:	1.0
double-arc precision:	1.0

Average evaluation scores
-------------------------
recall:	1.0
precision:	1.0
accuracy:	1.0


In [10]:
# Visualize effect of using insufficiently small model buffers
ocpn_buf_too_small = OcpnBuffer(
    5,
    5,
    5,
    CachePolicy.FIFO,
    use_mixed_ocdfg_buf=True
)
ocpn_buf_too_small.process_stream(event_stream.stream)
ocpn_model_too_small = OcpnModel(ocpn_buf_too_small)
ocpn_model_too_small.visualize(Path('../figures/SmallExample'), 'ocpn_too_small.pdf', ot_to_hex_color)

# Evaluate online model with insufficiently small model buffers to fully capture SmallExample
print_ocpn_scores(ocpn_model_too_small, ocpn_model_offl)

Total evaluation scores
-----------------------
act-place-act recall:	0.3333333333333333
act-place-act accuracy:	0.8461538461538461
act-place-act precision:	1.0
double-arc recall:	None
double-arc accuracy:	1.0
double-arc precision:	None

Average evaluation scores
-------------------------
recall:	0.3333333333333333
precision:	1.0
accuracy:	0.9230769230769231
